# Реализация эксперимента (нулевая версия агента), в котором агент получает информацию из интернета при помощи простого скрипта без введения специального инструмента и самостоятельного решения о его использовании

## Цель этого эксперимента: определить, насколько целесообразно давать возможность агенту самостоятельно ходить в поиск, а не делать это за него. Будем сравнивать эту версию с первой и второй, где был реализован поиск через инструмент

## Воспроизвести результаты, полученные в ходе тестирования агента, может быть затруднительно из-за ошибки превышения лимита запросов DuckDuckGo. Ближе к концу тестирования ошибка возникала на каждом примере, поэтому агент не получал какую-либо дополнительную информацию. В последующих версиях я хотел использовать тоже DuckDuckGo, но ошибка стала перманентной, попытки ее решить не увенчались успехом

In [1]:
!pip install -q langchain_core
!pip install -q langchain_community
!pip install -q langchain_openai
!pip install -q langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 

In [8]:
!pip install -U -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.8 MB/s eta 0:00:0000:0100:01


In [2]:
from langgraph.graph import END, StateGraph
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from typing import TypedDict, Annotated, Literal

In [3]:
API_KEY = "апи ключ в Google AI Studio"
MODEL = "gemini-2.5-flash" # в ситуациях, когда заканчивались бесплатные запросы в день, менял на gemini-2.0-flash

In [23]:
class AgentState(TypedDict):
    user_query: str
    search_result: Annotated[str, 'Ответ из сервиса "Карты"']
    relevance_score: Annotated[Literal[0, 1] | None, "Оценка релевантности"]
    search_data: Annotated[str | None, "Данные из интернета"]

In [24]:
llm = ChatOpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=OPENROUTER_API_KEY,
    model=OPENROUTER_MODEL,
    temperature=0
)

In [9]:
search = DuckDuckGoSearchRun()

In [10]:
analyst_prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты эксперт по оценке релевантности запросов в сервис "Карты" и ответов этого сервиса. Оцени, насколько ответ сервиса соответствует запросу.
Данные для анализа:
1. Запрос пользователя: {user_query}
2. Ответ поисковика: {search_result}
3. Дополнительные данные из интернета: {search_data}

Инструкции:
1. Проанализируй соответствие по тематике, местоположению, ценам и другим параметрам
2. Верни только 0 (нерелевантно) или 1 (релевантно) без пояснений"""),
    ("human", "Твоя оценка (0 или 1):")
])

def search_info(state: AgentState):
    query = f"{state['user_query']} {state['search_result'][0:state['search_result'].find('[Description]')]}"
    try:
        search_data = search.run(query)
        return {"search_data": search_data}
    except Exception as e:
        print("search_error")
        return {"search_data": "В интернете не удалось найти подробную информацию, придется тебе подумать самому"}

def analyze_relevance(state: AgentState):
    response = llm.invoke(analyst_prompt.format_messages(
        user_query=state['user_query'],
        search_result=state['search_result'],
        search_data=state.get('search_data', '')
    ))
    return {"relevance_score": int(response.content.strip())}

In [25]:
workflow = StateGraph(AgentState)
workflow.add_node("search", search_info)
workflow.add_node("analyze", analyze_relevance)
workflow.set_entry_point("search")
workflow.add_edge("search", "analyze")
workflow.add_edge("analyze", END)
chain = workflow.compile()

In [26]:
def run_evaluation(query: str, search_data: str):
    state = {
        "user_query": query,
        "search_result": search_data,
        "relevance_score": None,
        "search_data": None
    }
    result = chain.invoke(state)
    return result["relevance_score"]

In [2]:
import pandas as pd
data = pd.read_csv("/kaggle/input/eval_gemini_exp1_exp2.csv")
data # в ходе тестирования будем брать информацию из первых трех полей и записывать в поле gemini_2.5_flash_exp1
# так будет выглядеть датафрейм после тестирования (на другие поля не обращать внимания)

,request,response,target,gemini_2.5_flash_exp1,gemini_2.0_flash_exp2,baseline
0,сигары,"[Address]\n Москва, Дубравная улица, 34/29\n ...",1.0,1.0,1.0,1.0
1,кальянная спб мероприятия,"[Address]\n Санкт-Петербург, Большой проспект...",0.0,0.0,0.0,0.0
2,Эпиляция,"[Address]\n Московская область, Одинцово, ули...",1.0,1.0,1.0,1.0
3,стиральных машин,"[Address]\n Москва, улица Обручева, 34/63\n ...",1.0,1.0,1.0,1.0
4,сеть быстрого питания,"[Address]\n Санкт-Петербург, 1-я Красноармейс...",1.0,1.0,1.0,1.0
...,...,...,...,...,...,...
495,наращивание ресниц,"[Address]\n Саратов, улица имени А.С. Пушкина...",1.0,1.0,0.0,0.0
496,игры,"[Address]\n Москва, Щёлковское шоссе, 79, кор...",0.0,1.0,1.0,1.0
497,домашний интернет в курске что подключить отзы...,"[Address]\n Курск, Садовая улица, 5\n [Name]...",0.0,0.0,1.0,0.0
498,гостиница волгодонск сауна номер телефона,"[Address]\n Ростовская область, городской окр...",0.0,1.0,1.0,1.0


In [27]:
# иногда возникает ошибка превышения лимита запросов к модели в промежуток времени, в последней версии агента будут попытки ее исправить
# но пока будем просто запоминать номер примера, на котором возникла ошибка
# поэтому это код, актуальный для последнего случая продолжения после ошибки
from tqdm.auto import tqdm
for i in tqdm(range(492,len(data))):
    user_query = data.iloc[i,0]
    search_result = data.iloc[i,1]
    score = run_evaluation(user_query, search_result)
    data.iloc[i,3]=score
    print(data.iloc[i,2], data.iloc[i,3])

  0%|          | 0/8 [00:00<?, ?it/s]

search_error
0.0 0.0
1.0 1.0
1.0 1.0
search_error
1.0 1.0
search_error
0.0 1.0
search_error
0.0 0.0
search_error
0.0 1.0
search_error
0.0 0.0


In [9]:
# Посчитаем accuracy
ok=0
for i in range(500):
    if data.iloc[i,2]==data.iloc[i,3]:
        ok+=1
print(ok/500)

0.764


In [8]:
# Сравним с бейзлайном (колонка baseline)
ok_b=0
for i in range(500):
    if data.iloc[i,2]==data.iloc[i,5]:
        ok_b+=1
print(ok_b/500)

0.71


In [10]:
data.to_csv("eval_gemini_exp1_exp2.csv",index=False)

## Оценка проводилась на переразмеченной выборке. Эксперимент показал, что использование агента в данной задаче целесообразно, но можно получить качество лучше (даже с учетом оставшейся неоднозначности новых целевых меток)